In [1]:
import pandas as pd
import numpy as np

#para la extracción de datos
import requests  # Requests permite hacer peticiones HTTP para interactuar con APIs y sitios web.
import selenium
from selenium import webdriver  # WebDriver permite controlar un navegador de forma automatizada.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait  # WebDriverWait permite esperar hasta que una condición específica se cumpla.
from selenium.webdriver.support import expected_conditions as EC  # EC ayuda a definir condiciones para WebDriverWait.
from selenium.common.exceptions import NoSuchElementException  # Excepciones comunes de Selenium que nos podemos encontrar.
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup  # BeautifulSoup permite parsear HTML y extraer información del contenido de las páginas web.
from time import sleep
import re
import random

import sys 
import os
import warnings
warnings.filterwarnings("ignore")
sys.path.append(os.path.abspath("../src"))   
import soporte_extraccion as q
pd.set_option('display.max_columns', None)

Hago la extracción con selenium de pisos en la web de Red Piso

In [2]:
encabezados=["tamanio", "metros_utiles","dormitorios", "banios","calidad_energetica","estado","orientacion","ventanas","tipo_suelo","tipo_fachada","antiguedad","anio_construccion","precio"]

In [3]:
lista_prov=["Álava", 'Albacete', 'Alicante', "Almería", 'Ávila', "Badajoz", #habrá que cambiar Tenerife por Santa Cruz de Tenerife antes de pasar el modelo predictivo
       "Baleares", 'Barcelona', 'Burgos', "Cáceres", 'Cádiz', 'Castellón',
       "Ciudad Real", 'Córdoba', "A Coruña", 'Cuenca', "Girona",
       'Granada', 'Guadalajara', "Guipúzcoa", 'Huelva', 'Huesca', "Jaén",
       "León", 'Lleida', "La Rioja", "Lugo", 'Madrid', 'Málaga', 'Murcia',
       "Navarra", 'Ourense', "Asturias", 'Palencia',
       "Las Palmas de Gran Canaria", "Pontevedra", "Salamanca",
       "Tenerife", "Cantabria", 'Segovia', 'Sevilla',
       "Soria", "Tarragona", "Teruel", 'Toledo', 'Valencia', 'Valladolid',
       "Vizcaya", "Zamora", 'Zaragoza', "Ceuta", "Melilla"]

lista_provincias=[]
for prov in lista_prov:
      provincia=prov.replace(" ","-").lower()
      lista_provincias.append(provincia)


In [ ]:


columnas_globales = set()
driver = webdriver.Chrome()
df_final_prov=pd.DataFrame()
try:
    for provincia in lista_provincias:
        df_intermedio_prov = pd.DataFrame()
        for pag in range(1, 279):    #pongo 279 porque madrid que es la que más páginas tienen tienen 277 páginas
            url_red = f"https://www.redpiso.es/venta-viviendas/{provincia}/pagina-{pag}"
            sleep(random.uniform(2, 5))
            lista_links = []
            try:
                driver.get(url_red)    #lo inicio con selenium en vez de con requests porque causaba fallos
                html = driver.page_source
                sopa_pisos = BeautifulSoup(html, "html.parser")
                pisos = sopa_pisos.find_all("a", class_="item-link")

                if not pisos:
                    print(f"No se encontraron pisos en la página {pag} de la provincia {provincia}.")
                    break  #pasamos a la siguiente provincia
                
                df_intermedio_pag = pd.DataFrame()

                #obtengo los enlaces limpios de los anuncios
                for link_sucio in pisos:
                    links = link_sucio.get("href")
                    
                    lista_links.append(links)

                #uso regex para detectar las claves con :
                regex_clave_valor = r"([^:]+):\s*(.+)"     #siguiendo la estructura "clave: valor"

                for link in lista_links:
                    sleep(random.uniform(1.5, 4.5))  
                    driver.get(link)

                    caracteristicas_prueba = driver.find_element("css selector", "body > section:nth-child(13) > div > div > div:nth-child(2)").text
                    precio_prueba = driver.find_element("css selector", "body > section:nth-child(7) > div > div > div > div > div > div > div > div.property-contact-item > h2").text

                    #proceso el precio
                    coste = precio_prueba.replace(".", "").replace(" €", "")

                    #proceso las características
                    datos = caracteristicas_prueba.replace(" m²", "").split("\n")
                    datos.append(f"Precio: {coste}")
                    caracteristicas_dict = {}

                    for dato in datos:
                        match = re.match(regex_clave_valor, dato)
                        if match:
                            clave = match.group(1).strip()
                            valor = match.group(2).strip()
                            caracteristicas_dict[clave] = valor
                        else:
                            caracteristicas_dict[dato.strip()] = "Si"

                    #genero el df del dicc
                    df_anuncio = pd.DataFrame([caracteristicas_dict])
                    df_anuncio["provincia"] = provincia

                    #actualizo la lista de columnas únicas
                    columnas_globales.update(df_anuncio.columns)

                    #coloco los valores en las columnas donde correspondan rellenando con None
                    df_anuncio = df_anuncio.reindex(columns=columnas_globales, fill_value=None)

                    #concateno al dataframe intermedio_pag
                    df_intermedio_pag = pd.concat([df_intermedio_pag, df_anuncio], ignore_index=True)

                #concateno al dataframe intermedio_prov
                df_intermedio_prov = pd.concat([df_intermedio_prov, df_intermedio_pag], ignore_index=True)

                #concateno al dataframe final_prov
                df_final_prov = pd.concat([df_final_prov, df_intermedio_prov], ignore_index=True)
                
            except Exception as e:
                print(f"Error en página {pag} de {provincia}: {e}")
                break  # para que pase a la siguiente provincia

        #guardamos los datos al final de cada provincia
        df_intermedio_prov.to_csv(f"../datos/red_piso/Datos_extraccion/provincias_completas/datos_{provincia}.csv")

finally:
    # cierro el driver siempre al final
    df_final_prov.to_csv(f"../datos/red_piso/Datos_extraccion/csv_unico/datos_provincias_unico.csv")
    driver.quit()


In [2]:
df_unico=pd.read_csv("../datos/red_piso/Datos_extraccion/csv_unico/datos_provincias_unico.csv", index_col=0)

In [3]:
df_unico.shape

(352116, 51)

Tras haber investigado, los duplicados nos damos cuenta que surgen porque realmente en el proceso de extracción los duplico pero se soluciona con un drop duplicates

In [4]:
df_unico[df_unico.duplicated()].shape

(348231, 51)

In [5]:
df_fin=df_unico.drop_duplicates()
df_fin.shape

(3885, 51)

In [6]:
df_fin.reset_index(drop=True, inplace=True)

In [7]:
df_fin.head()

,Ascensor,Metros,Año de construcción,Acceso minusválidos,Metros útiles,Dormitorios,Agua caliente,provincia,Baños,Exterior,Trastero,Orientación,Amueblado,Calefacción,Precio,Antigüedad,Cocina equipada,Número de terrazas,Garajes,Tipo de suelo,Tipo de fachada,Circuito cerrado de seguridad,Puerta de seguridad,Aire acondicionado,Estado,Permite mascotas,Cal. energética:,Comunidad,Piscina,Armarios,Alarma,Plantas,Zonas verdes,Pareado,Pisos,Individual,Pista de tenis,Conserje,Pista de fútbol,Adosado,Pista de baloncesto,Consumo,Inquilinos,Emisiones,Campo de golf,Pista de pádel,Cal. energética,Pista de squash,Gimnasio,Salida de humos,Unnamed: 51
0,Si,142.0,1974.0,Si,120.0,3.0,Central,albacete,2.0,Si,Si,Oeste,Si,Central,249000,Mas de 25 Años,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Si,137.0,1974.0,Si,136.0,3.0,Individual,albacete,2.0,Si,NaN,Noreste,Si,Individual,250000,NaN,Si,1.0,1.0,Parquet,Piedra,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Si,96.0,1987.0,Si,95.0,3.0,Individual,albacete,2.0,NaN,NaN,NaN,Si,Individual,A consultar,NaN,Si,NaN,NaN,Parquet,Ladrillo,Si,Si,Si,Perfecto Estado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,50.0,NaN,Si,40.0,1.0,Individual,alicante,1.0,Si,NaN,NaN,Si,Individual,85000,Entre 35 y 50 Años,Si,NaN,NaN,Gres,Cemento,NaN,NaN,Si,Perfecto Estado,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,64.0,1968.0,NaN,58.0,3.0,Individual,alicante,1.0,Si,Si,Sureste,NaN,Eléctrica,105000,Más de 50 años,NaN,1.0,NaN,Gres,Cemento,NaN,Si,NaN,Perfecto Estado,NaN,Si,"30,00 €",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_fin.to_csv("../datos/red_piso/Datos_extraccion/datos_provincias_sin_duplicados.csv")

### Siguientes pasos:  
  
- **Los NaN que correspondan a opciones de Si igual hay que pasarlos a No**  

- **Eliminar las columnas que correspondan**      
  
- **Tratar las columnas que correspondan**       

In [ ]:
df_fin[df_fin["Unnamed: 51"]== "Si"]  #esta nos la tendremos que cargar por ejemplo

,Ascensor,Metros,Año de construcción,Acceso minusválidos,Metros útiles,Dormitorios,Agua caliente,provincia,Baños,Exterior,Trastero,Orientación,Amueblado,Calefacción,Precio,Antigüedad,Cocina equipada,Número de terrazas,Garajes,Tipo de suelo,Tipo de fachada,Circuito cerrado de seguridad,Puerta de seguridad,Aire acondicionado,Estado,Permite mascotas,Cal. energética:,Comunidad,Piscina,Armarios,Alarma,Plantas,Zonas verdes,Pareado,Pisos,Individual,Pista de tenis,Conserje,Pista de fútbol,Adosado,Pista de baloncesto,Consumo,Inquilinos,Emisiones,Campo de golf,Pista de pádel,Cal. energética,Pista de squash,Gimnasio,Salida de humos,Unnamed: 51
1592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,madrid,NaN,NaN,NaN,NaN,NaN,NaN,A consultar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Si
